# Match tensor
## NOTE: no where close to fully functional.

In [1]:
import json
import os
import sys
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing import sequence
from keras.layers import merge, Dense, Input,Dropout, Embedding, LSTM, Bidirectional, Activation
from keras.layers import Conv2D
from keras.models import Model
from HomeDepotCSVReader import HomeDepotReader
import Utilities
from DataPreprocessing import DataPreprocessing
from Feature_Word2Vec import Feature_Word2Vec

Using TensorFlow backend.


In [2]:
train_filename = '../data/train_play.csv'
test_filename = '../data/test_play.csv'
attribute_filename = '../data/attributes_play.csv'
description_filename = '../data/product_descriptions_play.csv'
word2vec_model_path='model/word2vec_play.model'
vocab_path='model/word2vec_play_vocab.json'
embeddings_path='model/embeddings_play.npz'

# train_filename = '../data/train.csv'
# test_filename = '../data/test.csv'
# attribute_filename = '../data/attributes.csv'
# description_filename = '../data/product_descriptions.csv'
# word2vec_model_path='model/word2vec.model'
# vocab_path='model/word2vec_vocab.json'
# embeddings_path='model/embeddings.npz'

In [3]:
reader = HomeDepotReader()

train_query_df, product_df, attribute_df, test_query_df = reader.getQueryProductAttributeDataFrame(train_filename,
                                              test_filename,
                                              attribute_filename,
                                              description_filename)
print("train_query_df:",list(train_query_df))
print("product_df:", list(product_df))
print("attribute_df:", list(attribute_df))
print("test_query_df:", list(test_query_df))

===========Tranforming labels...
showing current values
trainDF: ['id', 'product_uid', 'search_term', 'relevance']
trainDF:    id  product_uid    search_term  relevance
0   2       100001  angle bracket        3.0
self.mergedLabelDF: ['relevance'] 
 <class 'pandas.core.frame.DataFrame'> (8, 1)    relevance
0        3.0
Old unique Labels: [ 2.33  2.5   2.67  3.  ]
newLabels: [0 1 2 3]
Creating new column for training:  relevance_int
===========Transform labels completed
train_query_df: ['id', 'product_uid', 'search_term', 'relevance', 'relevance_int', 'product_idx']
product_df: ['product_title', 'product_uid', 'product_description']
attribute_df: ['product_uid', 'name', 'value']
test_query_df: ['id', 'product_uid', 'search_term']


In [4]:
#transform attribute into doc
dp = DataPreprocessing()
attribute_doc_df = dp.getAttributeDoc(attribute_df)
#attribute_doc_df
product_df=product_df.join(attribute_doc_df.set_index('product_uid'), on = 'product_uid')

In [5]:
product_df['content'] = product_df['product_title'].map(str) + " " + \
                        product_df['product_description'].map(str) + " " + \
                        product_df['attr_json'].map(str)
        
# ## no attribute
# product_df['content'] = product_df['product_title'].map(str) + " " + \
#                         product_df['product_description'].map(str) 
        
product_df['content']

0    Simpson Strong-Tie 12-Gauge Angle Not only do ...
1    BEHR Premium Textured DeckOver 1-gal. #SC-141 ...
2    Delta Vero 1-Handle Shower Only Faucet Trim Ki...
3    Whirlpool 1.9 cu. ft. Over the Range Convectio...
Name: content, dtype: object

In [6]:

w2v=Feature_Word2Vec(modelFilename=word2vec_model_path)
sentences=w2v.convertDFIntoSentences(product_df,'content')
#print(sentences)
w2v.trainModel(sentences)

Loading Word2Vec model
Initialise/Train Word2Vec model
('Time taken: ', 0, ' secs')
Saving vectors to disk
('Time taken: ', 0, ' secs')


In [7]:
#w2v.model.wv.vocab

In [8]:
print(w2v.getVectorFromWord('wood'))
print(w2v.getSimilarWordVectors('wood',5))
print(len(w2v.getVectorFromWord('wood')))

[  4.39748587e-03  -4.59543662e-03   2.49057752e-03   3.60534922e-03
  -2.38019810e-03   4.06930828e-03  -2.67027761e-03  -4.66374727e-03
   3.64948972e-03   4.73050540e-03   4.05373937e-03   3.57443443e-03
   4.26247763e-03   3.43804527e-03  -3.13495588e-03  -2.91322678e-04
  -1.73238001e-03  -3.92852956e-03  -3.71308601e-03  -3.26616759e-03
   2.08726944e-03   2.94579938e-03  -1.19030941e-03   1.27816992e-03
   4.69220430e-03  -2.72117741e-03   4.96241963e-03   3.35770706e-03
  -1.85807655e-03  -5.19208238e-03  -4.44383593e-03  -2.32875906e-03
   1.50145311e-03  -4.83125262e-03   2.06220429e-03  -2.43607000e-03
  -3.38598201e-03   2.29467335e-03   2.38132942e-03   4.06370265e-03
   5.48644166e-04  -6.11748037e-05  -8.98292870e-04  -1.57675601e-03
  -3.61116906e-03  -4.53049364e-03   2.87177740e-04  -1.91700982e-03
   3.72708659e-04   4.00600303e-03   4.19722823e-03  -1.39494997e-03
  -3.38747050e-03   3.83756566e-03  -1.33652473e-03   2.14475556e-03
   4.26710304e-03   2.53871456e-03

# embeddings to keras
http://ben.bolte.cc/resources/embeddings/embeddings.html

In [9]:
vocab = dict([(k, v.index) for k, v in w2v.model.wv.vocab.items()])
with open(vocab_path, 'w') as f:
    f.write(json.dumps(vocab))

In [10]:
weights = w2v.model.wv.syn0
np.save(open(embeddings_path, 'wb'), weights)

In [11]:
def load_vocab(vocab_path):
    """
    Load word -> index and index -> word mappings
    :param vocab_path: where the word-index map is saved
    :return: word2idx, idx2word
    """

    with open(vocab_path, 'r') as f:
        data = json.loads(f.read())
    word2idx = data
    idx2word = dict([(v, k) for k, v in data.items()])
    return word2idx, idx2word

In [12]:
word2idx, idx2word = load_vocab(vocab_path)

In [13]:
def word2vec_embedding_layer(embeddings_path):
    """
    Generate an embedding layer word2vec embeddings
    :param embeddings_path: where the embeddings are saved (as a numpy file)
    :return: the generated embedding layer
    """

    weights = np.load(open(embeddings_path, 'rb'))
    layer = Embedding(input_dim=weights.shape[0], output_dim=weights.shape[1], weights=[weights])
    return layer

In [14]:
print_tokens=True

In [15]:
# variable arguments are passed to gensim's word2vec model
# if options.train:
#     print('Training Word2Vec...')
#     create_embeddings(options.data, options.embeddings, options.vocab, size=100, min_count=5, window=5, sg=1, iter=25)

word2idx, idx2word = load_vocab(vocab_path)

if print_tokens:
    print('Tokens:', ', '.join(word2idx.keys()))



Tokens: a, variety, of, in, and, to, or, is, can, be, the, for, at, x, from, with, 'Number, 'Product, Depth, (in, )':, Height, Width, BEHR, It, your, wood, concrete, ft, 'Assembled, in',, 'Yes',, Type':, on, not, that, Sensor, this, microwave, cooking, have, you, time, interior, fast, fan


# convert to idx

In [16]:
def query_sent2idx(df,col):
    query_word2vec_idx_list=[]
    queries=w2v.convertDFIntoSentences(df,col)
    print(len(queries))
    for query in queries:
        idx_list = []
        for word in query:
            if word not in word2idx.keys():
                idx_list+=[len(word2idx.keys())] # use last as special key
            else:
                idx_list+=[word2idx[word]]
        query_word2vec_idx_list+=[idx_list]
        print("=====")
        print(idx_list)
        print("=====")
    return query_word2vec_idx_list


In [17]:
query_word2vec_idx_list = query_sent2idx(train_query_df,'search_term')
print(query_word2vec_idx_list)

8
=====
[46, 46]
=====
=====
[46, 46]
=====
=====
[46, 46]
=====
=====
[46, 46, 46]
=====
=====
[46, 46, 46]
=====
=====
[46, 46]
=====
=====
[22, 46, 46]
=====
=====
[46]
=====
[[46, 46], [46, 46], [46, 46], [46, 46, 46], [46, 46, 46], [46, 46], [22, 46, 46], [46]]


In [18]:
# def doc_sent2idx(df,col):
#     doc_word2vec_idx_list=[]
#     for row in df.iteritems():
#         a=w2v.convertDFIntoSentences(row,col)
#         print(a)
# #    print(len(queries))
# #     for query in queries:
# #         idx_list = []
# #         for word in query:
# #             if word not in word2idx.keys():
# #                 idx_list+=[len(word2idx.keys())] # use last as special key
# #             else:
# #                 idx_list+=[word2idx[word]]
# #         query_word2vec_idx_list+=[idx_list]
# #         print("=====")
# #         print(idx_list)
# #         print("=====")
#     return doc_word2vec_idx_list

In [19]:
#TODO: this is fucked. just bodge for testing nn
doc_word2vec_idx_list = query_sent2idx(product_df,'product_title')
print(doc_word2vec_idx_list)

8
=====
[46, 46, 46, 46]
=====
=====
[40, 46, 46, 46, 46]
=====
=====
[46, 46, 46, 0, 46, 46]
=====
=====
[46, 46, 46, 46, 46, 46, 46, 46, 6, 46, 46, 46, 46]
=====
=====
[46, 46]
=====
=====
[46, 46]
=====
=====
[18]
=====
=====
[46, 1, 46, 46, 46, 6, 46, 46, 7, 33, 46]
=====
[[46, 46, 46, 46], [40, 46, 46, 46, 46], [46, 46, 46, 0, 46, 46], [46, 46, 46, 46, 46, 46, 46, 46, 6, 46, 46, 46, 46], [46, 46], [46, 46], [18], [46, 1, 46, 46, 46, 6, 46, 46, 7, 33, 46]]


In [20]:
# # cosine similarity model
# print('Building model...')
# input_a = Input(shape=(1,), dtype='int32', name='input_a')
# input_b = Input(shape=(1,), dtype='int32', name='input_b')
# embeddings = word2vec_embedding_layer(embeddings_path)
# embedding_a = embeddings(input_a)
# embedding_b = embeddings(input_b)
# similarity = merge([embedding_a, embedding_b], mode='cos', dot_axes=2)
# model = Model(input=[input_a, input_b], output=similarity)
# model.compile(optimizer='sgd', loss='mse') # optimizer and loss don't matter


# word_a = 'wood'#raw_input('First word: ')
# if word_a not in word2idx:
#     print('"%s" is not in the index' % word_a)
# word_b = 'fan'#raw_input('Second word: ')
# if word_b not in word2idx:
#     print('"%s" is not in the index' % word_b)
# output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
# print('%f' % output)

# Define model

In [21]:
query_max_len = 6 #covers 95.74% of the search lengths (see data exploration)
doc_max_len = 400 # todo: confirm this is sensible

In [34]:
print('Building model...')
embeddings = word2vec_embedding_layer(embeddings_path)

#embedding lookup
input_query = Input(shape=(query_max_len,), dtype='int32', name='input_query')
#input_doc = Input(shape=(doc_max_len,), dtype='int32', name='input_doc')
embedding_query = embeddings(input_query)  # (None, 6, 100)
#embedding_doc = embeddings(input_doc)      # (None, 400, 100)

#shared linear projection
shared_lp = Dense(40,activation='linear')
query_output = shared_lp(embedding_query) # (None, 6, 40) 
#doc_output = shared_lp(embedding_doc) #(None, 400, 40) 

#TODO: fix output shape here, not right. should be (None, 6, 30) for query
#query: bi LSTM, lp
query_output = Bidirectional(LSTM(30))(query_output) #(None, 60) #bidirectional x2
query_output = Dense(50,activation='linear')(query_output) #(None, 50)

#doc: bi LSTM, lp
#doc_output = Bidirectional(LSTM(140))(doc_output) #(None, 280) #bidirectional x2
#doc_output = Dense(50,activation='linear')(doc_output) #(None, 50)

#2d product
#output = merge([query_output, doc_output], mode='mul') #(None, 50) #TODO: verify this shape

#Append exact match channel
#TODO:

# Conv layer

# Final layer
#output = Dense(1,activation='sigmoid')(output)

# build model
#model = Model([input_query], [encoded_query])
#model = Model([input_doc], [encoded_doc])
#model = Model([input_query,input_doc], [output])
model = Model([input_query,input_doc], [query_output])
#model = Model([input_query,input_doc], [query_output,doc_output])


model.summary()

Building model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_query (InputLayer)         (None, 6)             0                                            
____________________________________________________________________________________________________
embedding_5 (Embedding)          (None, 6, 100)        4600        input_query[0][0]                
____________________________________________________________________________________________________
dense_11 (Dense)                 (None, 6, 40)         4040        embedding_5[0][0]                
____________________________________________________________________________________________________
bidirectional_5 (Bidirectional)  (None, 60)            17040       dense_11[0][0]                   
_________________________________________________________________________

In [29]:
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

In [30]:
x_query_train=np.array(query_word2vec_idx_list)
x_doc_train=np.array(doc_word2vec_idx_list)

print("Pad sequences (samples x time)")
x_query_train = sequence.pad_sequences(x_query_train, maxlen=query_max_len)
x_doc_train = sequence.pad_sequences(x_doc_train, maxlen=doc_max_len)
# x_query_test = sequence.pad_sequences(x_query_test, maxlen=query_max_len)
# x_doc_test = sequence.pad_sequences(x_doc_test, maxlen=doc_max_len)

print('x_query_train shape:', x_query_train.shape)
print('x_doc_train shape:', x_doc_train.shape)
# print('x_query_test shape:', x_query_test.shape)
# print('x_doc_test shape:', x_doc_test.shape)

y_train=train_query_df['relevance_int'].as_matrix()
# y_test=test_query_df['relevance_int'].as_matrix()


Pad sequences (samples x time)
x_query_train shape: (8, 6)
x_doc_train shape: (8, 400)


In [31]:
batch_size=2
print('Train...')
hist=model.fit([x_query_train,x_doc_train], [y_train],
          batch_size=batch_size,
          #epochs=4,
          #validation_data=[x_test, y_test])
         )


Train...


ValueError: Error when checking model target: expected embedding_2 to have 3 dimensions, but got array with shape (8, 1)

In [26]:
model.predict([x_query_train,x_doc_train],
          batch_size=batch_size,
          #epochs=4,
          #validation_data=[x_test, y_test])
         )

array([[ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.],
       [ 1.]], dtype=float32)

In [27]:
print("Predict")
word_a = 'wood'#raw_input('First word: ')
if word_a not in word2idx:
    print('"%s" is not in the index' % word_a)
word_b = 'fan'#raw_input('Second word: ')
if word_b not in word2idx:
    print('"%s" is not in the index' % word_b)
output = model.predict([np.asarray([word2idx[word_a]]), np.asarray([word2idx[word_b]])])
print('%f' % output)

Predict


ValueError: Error when checking : expected input_query to have shape (None, 6) but got array with shape (1, 1)